In [1]:
import re
import sys
import time
from pyspark import SparkConf, SparkContext
from itertools import combinations
st = time.time()
conf = SparkConf()
sc = SparkContext(conf=conf)
lines = sc.textFile("soc-data.txt")
user_friend = lines.map(lambda l:tuple(l.split('\t')))
user_ratingprod = user_friend.flatMap(lambda x:[((x[0],i),-1000) for i in x[1].split(',')])
u_friend = user_friend.map(lambda x:(x[1].split(',')))
user_ratingprod2 = u_friend.flatMap(lambda w:[(i,1) for i in combinations(w,2)])
unio = user_ratingprod.union(user_ratingprod2).reduceByKey(lambda n1,n2:n1+n2).map(lambda x: (x[0][0],(x[0][0],x[0][1],x[1])))
result = unio.filter(lambda u:u[1][2] >= 0).groupByKey().map(lambda v: sorted(v[1], key=lambda k: k[2], reverse=True)).map(lambda x: x[0:10]).collect()
sc.stop()
end = time.time()
print(end-st)



172.84593081474304


In [2]:
result

[[('9696', '25195', 4),
  ('9696', '29490', 3),
  ('9696', '29379', 3),
  ('9696', '29516', 3),
  ('9696', '25225', 3),
  ('9696', '10026', 3),
  ('9696', '10812', 2),
  ('9696', '14052', 2),
  ('9696', '22504', 2),
  ('9696', '20741', 2)],
 [('7956', '22265', 4),
  ('7956', '16251', 3),
  ('7956', '22158', 3),
  ('7956', '22291', 3),
  ('7956', '16201', 3),
  ('7956', '34825', 3),
  ('7956', '15843', 2),
  ('7956', '18916', 2),
  ('7956', '22303', 2),
  ('7956', '22294', 2)],
 [('18928', '36821', 3),
  ('18928', '34644', 3),
  ('18928', '23557', 2),
  ('18928', '36676', 2),
  ('18928', '23281', 2),
  ('18928', '36732', 2),
  ('18928', '36685', 2),
  ('18928', '23270', 2),
  ('18928', '36820', 2),
  ('18928', '30220', 2)],
 [('14014', '14027', 35),
  ('14014', '14015', 32),
  ('14014', '14140', 31),
  ('14014', '14040', 29),
  ('14014', '14055', 25),
  ('14014', '14138', 25),
  ('14014', '14125', 25),
  ('14014', '14064', 24),
  ('14014', '14044', 24),
  ('14014', '14043', 23)],
 [('14

In [3]:
len(a.collect())

48353

In [6]:
st = time.time()
lines = sc.textFile('soc-data.txt')
user_friend = lines.map(lambda l:tuple(l.split('\t')))
user_ratingprod = user_friend.flatMap(lambda x:[((x[0],i),-1000) for i in x[1].split(',')])
u_friend = user_friend.map(lambda x:(x[1].split(',')))
user_ratingprod2 = u_friend.flatMap(lambda w:[(i,1) for i in combinations(w,2)])
#user_ratingprod2 = user_friend.flatMap(lambda x: (lambda j:[[((j[u],j[i]),1) for i in range(u+1,len(j))]for u in range(0,len(j))])(x[1].split(',')))
#user_ratingprod2 = user_ratingprod2.flatMap(lambda xs: [x for x in xs])
unio = user_ratingprod.union(user_ratingprod2).reduceByKey(lambda n1,n2:n1+n2)
unio = unio.map(lambda x: (x[0][0],(x[0][0],x[0][1],x[1])))
unio_f = unio.filter(lambda u:u[1][2] >= 0)
a = unio_f.groupByKey().map(lambda v: sorted(v[1], key=lambda k: k[2], reverse=True))

result = a.map(lambda x: x[0:10]).collect()
end = time.time()
print(end-st)

AttributeError: 'NoneType' object has no attribute 'sc'

In [97]:
len(result[0])

10

In [18]:
lines = sc.textFile('soc-short.txt')
user_friend = lines.map(lambda l:tuple(l.split('\t')))
user_ratingprod = user_friend.flatMap(lambda x:[((x[0],i),-1000) for i in x[1].split(',')])
u_friend = user_friend.map(lambda x:(x[1].split(',')))
user_ratingprod2 = u_friend.flatMap(lambda w:[(i,1) for i in combinations(w,2)])
#user_ratingprod2 = user_friend.flatMap(lambda x: (lambda j:[[((j[u],j[i]),1) for i in range(u+1,len(j))]for u in range(0,len(j))])(x[1].split(',')))
#user_ratingprod2 = user_ratingprod2.flatMap(lambda xs: [x for x in xs])
unio = user_ratingprod.union(user_ratingprod2).reduceByKey(lambda n1,n2:n1+n2)
unio = unio.map(lambda x: (x[0][0],(x[0][1],x[1])))

In [20]:
unio.groupByKey().map(lambda rec: sorted(rec[1], key=lambda k: float(k.split(",")[4]), reverse=True)).
take(100)

[('4', <pyspark.resultiterable.ResultIterable at 0x7fdac469ac18>),
 ('10', <pyspark.resultiterable.ResultIterable at 0x7fdac46a2438>),
 ('12', <pyspark.resultiterable.ResultIterable at 0x7fdac46a37f0>),
 ('29418', <pyspark.resultiterable.ResultIterable at 0x7fdac469a9b0>),
 ('16', <pyspark.resultiterable.ResultIterable at 0x7fdac46a4ef0>),
 ('20', <pyspark.resultiterable.ResultIterable at 0x7fdac46a7588>),
 ('24', <pyspark.resultiterable.ResultIterable at 0x7fdac46ab518>),
 ('26', <pyspark.resultiterable.ResultIterable at 0x7fdac46a3a58>),
 ('40', <pyspark.resultiterable.ResultIterable at 0x7fdac46ae518>),
 ('44', <pyspark.resultiterable.ResultIterable at 0x7fdac46b0080>),
 ('50', <pyspark.resultiterable.ResultIterable at 0x7fdac46b0cf8>),
 ('53', <pyspark.resultiterable.ResultIterable at 0x7fdac46b0a90>),
 ('54', <pyspark.resultiterable.ResultIterable at 0x7fdac46b0eb8>),
 ('56', <pyspark.resultiterable.ResultIterable at 0x7fdac46b0e48>),
 ('57', <pyspark.resultiterable.ResultIterable

In [45]:
import re
import sys
from pyspark import SparkConf, SparkContext
from itertools import combinations
#conf = SparkConf()
#sc = SparkContext(conf=conf)
lines = sc.textFile('soc-short.txt')
user_friend = lines.map(lambda l:tuple(l.split('\t')))
user_ratingprod = user_friend.flatMap(lambda x:[((x[0],i),-1000) for i in x[1].split(',')])
u_friend = user_friend.map(lambda x:(x[1].split(',')))
user_ratingprod2 = u_friend.flatMap(lambda w:[(i,1) for i in combinations(w,2)])
#user_ratingprod2 = user_friend.flatMap(lambda x: (lambda j:[[((j[u],j[i]),1) for i in range(u+1,len(j))]for u in range(0,len(j))])(x[1].split(',')))
#user_ratingprod2 = user_ratingprod2.flatMap(lambda xs: [x for x in xs])
unio = user_ratingprod.union(user_ratingprod2).reduceByKey(lambda n1,n2:n1+n2)
unio = unio.map(lambda x: (x[0][0],(x[0][0],x[0][1],x[1])))

In [28]:
unio.collect()

[('0', ('4', -999)),
 ('0', ('10', -999)),
 ('0', ('12', -998)),
 ('0', ('16', -998)),
 ('0', ('20', -998)),
 ('0', ('24', -1000)),
 ('0', ('26', -1000)),
 ('0', ('40', -1000)),
 ('0', ('44', -1000)),
 ('0', ('50', -1000)),
 ('0', ('53', -1000)),
 ('0', ('54', -1000)),
 ('0', ('56', -1000)),
 ('0', ('57', -1000)),
 ('0', ('60', -1000)),
 ('0', ('64', -1000)),
 ('0', ('70', -1000)),
 ('0', ('74', -999)),
 ('0', ('77', -999)),
 ('0', ('82', -1000)),
 ('0', ('83', -998)),
 ('0', ('86', -1000)),
 ('0', ('88', -1000)),
 ('1', ('20', -998)),
 ('1', ('8932', -1000)),
 ('1', ('10623', -1000)),
 ('1', ('12846', -1000)),
 ('1', ('23520', -1000)),
 ('1', ('32489', -1000)),
 ('1', ('38737', -1000)),
 ('1', ('44887', -1000)),
 ('1', ('623', -1000)),
 ('1', ('629', -1000)),
 ('1', ('14248', -1000)),
 ('1', ('15190', -1000)),
 ('1', ('19217', -1000)),
 ('1', ('20074', -1000)),
 ('1', ('29726', -1000)),
 ('1', ('33060', -1000)),
 ('1', ('34280', -1000)),
 ('1', ('49592', -1000)),
 ('2', ('117', -1000)

In [23]:
for i in unio.groupByKey().collect():
    print (i)

('4', <pyspark.resultiterable.ResultIterable object at 0x7fdac44eaba8>)
('10', <pyspark.resultiterable.ResultIterable object at 0x7fdac4600ac8>)
('12', <pyspark.resultiterable.ResultIterable object at 0x7fdac4609240>)
('29418', <pyspark.resultiterable.ResultIterable object at 0x7fdac44eab70>)
('16', <pyspark.resultiterable.ResultIterable object at 0x7fdac4605668>)
('20', <pyspark.resultiterable.ResultIterable object at 0x7fdac460abe0>)
('24', <pyspark.resultiterable.ResultIterable object at 0x7fdac460bc50>)
('26', <pyspark.resultiterable.ResultIterable object at 0x7fdac46091d0>)
('40', <pyspark.resultiterable.ResultIterable object at 0x7fdac460cc88>)
('44', <pyspark.resultiterable.ResultIterable object at 0x7fdac460f8d0>)
('50', <pyspark.resultiterable.ResultIterable object at 0x7fdac460d470>)
('53', <pyspark.resultiterable.ResultIterable object at 0x7fdac460d4e0>)
('54', <pyspark.resultiterable.ResultIterable object at 0x7fdac460d550>)
('56', <pyspark.resultiterable.ResultIterable obj

In [47]:
unio_f = unio.filter(lambda u:u[1][2] >= 0)
a = unio_f.groupByKey().map(lambda v: sorted(v[1], key=lambda k: k[2], reverse=True))
a.map(lambda x: x[0:10]).collect()

[('1', ('1', '4', 1)),
 ('1', ('1', '10', 1)),
 ('1', ('1', '12', 1)),
 ('1', ('1', '16', 1)),
 ('1', ('1', '24', 1)),
 ('1', ('1', '26', 1)),
 ('1', ('1', '40', 1)),
 ('1', ('1', '44', 1)),
 ('1', ('1', '50', 1)),
 ('1', ('1', '53', 1)),
 ('1', ('1', '54', 1)),
 ('1', ('1', '56', 1)),
 ('1', ('1', '57', 1)),
 ('1', ('1', '60', 1)),
 ('1', ('1', '64', 1)),
 ('1', ('1', '70', 1)),
 ('1', ('1', '74', 1)),
 ('1', ('1', '77', 1)),
 ('1', ('1', '82', 1)),
 ('1', ('1', '83', 1)),
 ('1', ('1', '86', 1)),
 ('1', ('1', '88', 1)),
 ('2', ('2', '5', 1)),
 ('2', ('2', '11', 1)),
 ('2', ('2', '13', 1)),
 ('2', ('2', '27', 1)),
 ('2', ('2', '28', 1)),
 ('2', ('2', '35', 1)),
 ('2', ('2', '37', 1)),
 ('2', ('2', '38', 1)),
 ('2', ('2', '39', 1)),
 ('2', ('2', '41', 1)),
 ('2', ('2', '46', 1)),
 ('2', ('2', '52', 1)),
 ('2', ('2', '55', 1)),
 ('2', ('2', '58', 1)),
 ('2', ('2', '72', 1)),
 ('2', ('2', '75', 1)),
 ('2', ('2', '76', 1)),
 ('2', ('2', '78', 1)),
 ('2', ('2', '79', 1)),
 ('2', ('2', '87',

In [60]:
a = unio_f.groupByKey().map(lambda rec: sorted(rec[1], key=lambda k: k[2], reverse=True))
a.collect()
#takeOrdered

[[('4', '85', 2),
  ('4', '83', 2),
  ('4', '38', 2),
  ('4', '46', 2),
  ('4', '9', 1),
  ('4', '17', 1),
  ('4', '19', 1),
  ('4', '21', 1),
  ('4', '22', 1),
  ('4', '29', 1),
  ('4', '33', 1),
  ('4', '34', 1),
  ('4', '45', 1),
  ('4', '48', 1),
  ('4', '63', 1),
  ('4', '66', 1),
  ('4', '68', 1),
  ('4', '69', 1),
  ('4', '73', 1),
  ('4', '84', 1),
  ('4', '91', 1),
  ('4', '93', 1),
  ('4', '6', 1),
  ('4', '7', 1),
  ('4', '23', 1),
  ('4', '25', 1),
  ('4', '30', 1),
  ('4', '31', 1),
  ('4', '32', 1),
  ('4', '36', 1),
  ('4', '42', 1),
  ('4', '43', 1),
  ('4', '47', 1),
  ('4', '49', 1),
  ('4', '51', 1),
  ('4', '59', 1),
  ('4', '61', 1),
  ('4', '62', 1),
  ('4', '65', 1),
  ('4', '67', 1),
  ('4', '71', 1),
  ('4', '81', 1),
  ('4', '94', 1),
  ('4', '10', 1),
  ('4', '12', 1),
  ('4', '16', 1),
  ('4', '20', 1),
  ('4', '24', 1),
  ('4', '26', 1),
  ('4', '40', 1),
  ('4', '44', 1),
  ('4', '50', 1),
  ('4', '53', 1),
  ('4', '54', 1),
  ('4', '56', 1),
  ('4', '57',

In [83]:
a = unio_f.groupByKey().map(lambda v: sorted(v[1], key=lambda k: k[2], reverse=True))
a.map(lambda x: x[0:10]).collect()

[[('4', '85', 2),
  ('4', '83', 2),
  ('4', '38', 2),
  ('4', '46', 2),
  ('4', '9', 1),
  ('4', '17', 1),
  ('4', '19', 1),
  ('4', '21', 1),
  ('4', '22', 1),
  ('4', '29', 1)],
 [('10', '29', 2),
  ('10', '85', 2),
  ('10', '83', 2),
  ('10', '38', 2),
  ('10', '41', 2),
  ('10', '55', 2),
  ('10', '89', 2),
  ('10', '14', 1),
  ('10', '17', 1),
  ('10', '19', 1)],
 [('12', '14', 1),
  ('12', '17', 1),
  ('12', '19', 1),
  ('12', '21', 1),
  ('12', '22', 1),
  ('12', '33', 1),
  ('12', '34', 1),
  ('12', '45', 1),
  ('12', '48', 1),
  ('12', '63', 1)],
 [('16', '30', 3),
  ('16', '29', 2),
  ('16', '85', 2),
  ('16', '83', 2),
  ('16', '38', 2),
  ('16', '41', 2),
  ('16', '55', 2),
  ('16', '89', 2),
  ('16', '17', 1),
  ('16', '19', 1)],
 [('20', '44846', 2),
  ('20', '30257', 2),
  ('20', '22939', 2),
  ('20', '35633', 2),
  ('20', '21', 1),
  ('20', '22', 1),
  ('20', '29', 1),
  ('20', '33', 1),
  ('20', '34', 1),
  ('20', '45', 1)],
 [('24', '29', 1),
  ('24', '33', 1),
  ('24

In [ ]:
map(lambda rec: sorted(rec[1], key=lambda k: float(k.split(",")[4]), reverse=True)).

In [ ]:
.filter(lambda u:u[2] >= 0).takeOrdered(10, key=lambda x: -x[2])

In [11]:
import re
import sys
import time
from pyspark import SparkConf, SparkContext
from itertools import combinations
conf = SparkConf()
sc = SparkContext(conf=conf)
st = time.time()
lines = sc.textFile('soc-data.txt')
user_friend = lines.map(lambda l:tuple(l.split('\t')))
user_ratingprod = user_friend.flatMap(lambda x:[((x[0],i),-1000) for i in x[1].split(',')])
u_friend = user_friend.map(lambda x:(x[1].split(',')))
user_ratingprod2 = u_friend.flatMap(lambda w:[(i,1) for i in combinations(w,2)])
#user_ratingprod2 = user_friend.flatMap(lambda x: (lambda j:[[((j[u],j[i]),1) for i in range(u+1,len(j))]for u in range(0,len(j))])(x[1].split(',')))
#user_ratingprod2 = user_ratingprod2.flatMap(lambda xs: [x for x in xs])
unio = user_ratingprod.union(user_ratingprod2).reduceByKey(lambda n1,n2:n1+n2)
unio = unio.map(lambda x: (x[0][0],(x[0][0],x[0][1],x[1])))
unio_f = unio.filter(lambda u:u[1][2] >= 0)
a = unio_f.groupByKey().map(lambda v: sorted(v[1], key=lambda k: k[2], reverse=True))

a.map(lambda x: x[0:10]).collect()
end = time.time()
print(end-st)



ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-10-309c0296399a>:7 

In [4]:
result.collect()

AttributeError: 'NoneType' object has no attribute 'setCallSite'